## AIS数据按每小时取平均，并且将所有数据合并到一个csv文件

In [1]:
import pandas as pd
import numpy as np
import movingpandas as mdp
from datetime import datetime


/usr/local/python/3.12.1/lib/python3.12/site-packages/movingpandas/__init__.py:37: UserWarning: Missing optional dependencies. To use the trajectory smoother classes please install Stone Soup (see https://stonesoup.readthedocs.io/en/latest/#installation).
  warnings.warn(e.msg, UserWarning)


In [2]:
# 新的数据容器
df_processed = pd.DataFrame(data=None,columns=['SOG','heading','draught', 'wind_val', 'wind_direction', 'wave_val', 'wave_direction', 'stream_val', 'stream_direction', 'postime', 'num'])

for shipnum in range(1, 14): # 13条船

    # 读取一条船跑满24小时的AIS数据
    df = pd.read_csv('processed_%d.csv' % shipnum,index_col=0)
    df['postime'] = pd.to_datetime(df['postime'],format='ISO8601')

    # 读取noon report，并且筛选出年月日
    df_fuel = pd.read_csv('../raw_data/mrv.csv')
    # 时间转换成datetime对象
    df_fuel['input_date'] = pd.to_datetime(df_fuel['input_date'],format='ISO8601')
    # 选择船的编号
    df_fuel = df_fuel[df_fuel['num'] == shipnum]
    # 筛选出一直在跑的船，这边考虑daily_voyage_time_mrv大于等于24小时的轨迹
    df_fuel = df_fuel[df_fuel['daily_voyage_time_mrv'] >= 24]
    # 删除关键字段存在空值的行，daily_fuel_mrv
    df_fuel = df_fuel.dropna(axis=0, subset = ['daily_fuel_mrv'])
    # 序号重新排列
    df_fuel = df_fuel.reset_index(drop=True)
    # 将日期全部抽取出来
    years = df_fuel['input_date'].dt.year
    months = df_fuel['input_date'].dt.month
    days = df_fuel['input_date'].dt.day

    # 临时数据容器，储存一条船3年的数据（经过每小时平均的）
    temp_data = pd.DataFrame(data=None,columns=['SOG','heading','draught', 'wind_val', 'wind_direction', 'wave_val', 'wave_direction', 'stream_val', 'stream_direction', 'postime', 'num'])
    for i in range(df_fuel.shape[0]): # 遍历一条船所有跑满24小时的天数
        for hours in range(24): #遍历每个小时内的点，并且做平均
            # 同一个小时的所有点
            temp_hours = df[(df['postime'].dt.year ==years[i]) & (df['postime'].dt.month ==months[i]) & (df['postime'].dt.day ==days[i]) & (df['postime'].dt.hour == hours)]
            avg = temp_hours[['SOG','heading','draught',  'wind_val', 'wind_direction', 'wave_val', 'wave_direction', 'stream_val', 'stream_direction']].mean()
            avg['postime'] = datetime(years[i], months[i], days[i], hours, 0, 0)
            avg['num'] = shipnum
            avg = avg.to_frame(); avg = avg.transpose()
            temp_data = pd.concat([temp_data, avg], ignore_index = True)
        temp_data = temp_data.dropna(axis=0)
    df_processed = pd.concat([df_processed, temp_data], ignore_index = True)

df_processed

,SOG,heading,draught,wind_val,wind_direction,wave_val,wave_direction,stream_val,stream_direction,postime,num
0,13.92,32.0,13.3,9.366,41.0,2.966,37.4,0.48,211.2,2021-01-04 00:00:00,1
1,13.95,31.5,13.3,9.71,40.0,2.99,37.0,0.51,206.0,2021-01-04 01:00:00,1
2,13.466667,32.333333,13.3,11.306667,36.0,2.923333,37.666667,0.646667,191.0,2021-01-04 02:00:00,1
3,13.3,37.666667,13.3,11.386667,36.666667,2.996667,37.666667,0.646667,184.333333,2021-01-04 03:00:00,1
4,13.533333,35.0,13.3,11.323333,36.666667,3.036667,37.333333,0.62,182.333333,2021-01-04 04:00:00,1
...,...,...,...,...,...,...,...,...,...,...,...
146404,17.684615,209.461538,13.7,2.736154,11.307692,0.505385,37.307692,0.090769,101.769231,2024-11-30 19:00:00,13
146405,17.72,203.6,13.7,6.772,37.8,1.376,94.6,0.446,260.0,2024-11-30 20:00:00,13
146406,18.233333,202.777778,13.7,7.278889,47.111111,1.438889,91.777778,0.578889,253.222222,2024-11-30 21:00:00,13
146407,17.78,198.6,13.7,4.704,28.8,0.882,54.0,0.342,147.6,2024-11-30 22:00:00,13


In [4]:
df_processed.to_csv('AIS_hour.csv')